In [65]:
import pandas as pd
import numpy as np
from datetime import date
from math import cos, pi

    
# Busy travel times (2010-2020)
trav_hol = ['2010-11-23', '2010-11-25', '2010-11-26', '2010-12-24',
            '2010-12-25', '2010-12-26',
            '2011-11-23', '2011-11-24', '2011-11-25', '2011-12-24',
            '2011-12-25', '2011-12-26',
            '2012-11-21', '2012-11-22', '2012-11-23', '2012-12-24',
            '2012-12-25', '2012-12-26',
            '2013-11-27', '2013-11-28', '2013-11-29', '2013-12-24',
            '2013-12-25', '2013-12-26',
            '2014-11-26', '2014-11-27', '2014-11-28', '2014-12-24',
            '2014-12-25', '2014-12-26',
            '2015-11-25', '2015-11-26', '2015-11-27', '2015-12-24',
            '2015-12-25', '2015-12-26',
            '2016-11-23', '2016-11-24', '2016-11-25', '2016-12-24',
            '2016-12-25', '2016-12-26',
            '2017-11-22', '2017-11-23', '2017-11-24', '2017-12-24',
            '2017-12-25', '2017-12-26',
            '2018-11-21', '2018-11-22', '2018-11-23', '2018-12-24',
            '2018-12-25', '2018-12-26',
            '2019-11-27', '2019-11-28', '2019-11-29', '2019-12-24',
            '2019-12-25', '2019-12-26',
            '2020-11-25', '2020-11-26', '2020-11-27', '2020-12-24',
            '2020-12-25', '2020-12-26' ]

# Dangerous holidays are New Years day and July 4th (2010-2020)
dang_hol = ['2010-01-01', '2010-07-04', '2011-01-01', '2011-07-04',
            '2012-01-01', '2012-07-04', '2013-01-01', '2013-07-04',
            '2014-01-01', '2014-07-04', '2015-01-01', '2015-07-04',
            '2016-01-01', '2016-07-04', '2017-01-01', '2017-07-04',
            '2018-01-01', '2018-07-04', '2019-01-01', '2019-07-04',
            '2020-01-01', '2020-07-04']

# Create empyt DataFrame
zeros = np.zeros((7,21))
columns = [u'mariners_home', u'seahawks_home', u'sounders_home', u'trav_holiday',
           u'dang_holiday', u'night', u'Monday', u'Saturday', u'Sunday',
           u'Thursday', u'Tuesday', u'Wednesday', u'day_num', u'zone1', u'zone2',
           u'zone3', u'zone4', u'zone5', u'zone6', u'zone7', u'seasonality',]
X_test = pd.DataFrame(zeros, columns=columns)

def query_to_X(query):
    # Set zones
    X_test.ix[0, 'zone1'] = 1
    X_test.ix[1, 'zone2'] = 1
    X_test.ix[2, 'zone3'] = 1
    X_test.ix[3, 'zone4'] = 1
    X_test.ix[4, 'zone5'] = 1
    X_test.ix[5, 'zone6'] = 1
    X_test.ix[6, 'zone7'] = 1
    
    #Set home games:
    if query['home_game'] == 'mariners':
        X_test['mariners_home'] = 1
    if query['home_game'] == 'seahawks':
        X_test['seahawks_homes'] = 1
    if query['home_game'] == 'sounders':
        X_test['sounders_home'] = 1
        
    # Set holidays
    X_test['trav_holiday'] = int(query['date_input'] in trav_hol)
    X_test['dang_holiday'] = int(query['date_input'] in dang_hol)

    # Set night:
    X_test['night'] = query['time_range']
    
    # Set weekday
    date_input = pd.to_datetime(query['date_input'])
    weekday = date_input.weekday_name
    if weekday in columns:
        X_test[weekday] = 1
        
    # Set day_num using timedelta with earliest date in dataset
    day0 = date(2010,6,29)
    X_test['day_num'] = (date_input.date() - day0).days
    
    # Set seasonality
    f = lambda x: 1 + cos(((2 * pi) / 365.25) * (x - 35))
    X_test['seasonality'] = X_test.day_num.apply(f)
    
    return X_test
 

In [68]:
query = {'date_input': '2016-11-23', 'num_units': 25, 'home_game': 'mariners', 'time_range': 0}



In [69]:
df = query_to_X(query)
df

,mariners_home,seahawks_home,sounders_home,trav_holiday,dang_holiday,night,Monday,Saturday,Sunday,Thursday,...,Wednesday,day_num,zone1,zone2,zone3,zone4,zone5,zone6,zone7,seasonality
0,1,0.0,0.0,1,0,0,0.0,1,0.0,0.0,...,1,2339,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.64354
1,1,0.0,0.0,1,0,0,0.0,1,0.0,0.0,...,1,2339,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.64354
2,1,0.0,0.0,1,0,0,0.0,1,0.0,0.0,...,1,2339,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.64354
3,1,0.0,0.0,1,0,0,0.0,1,0.0,0.0,...,1,2339,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.64354
4,1,0.0,0.0,1,0,0,0.0,1,0.0,0.0,...,1,2339,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.64354
5,1,0.0,0.0,1,0,0,0.0,1,0.0,0.0,...,1,2339,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.64354
6,1,0.0,0.0,1,0,0,0.0,1,0.0,0.0,...,1,2339,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.64354
